In [ ]:
import os
import os
import numpy as np
import pickle
import re
import time
datadir = '04-crypto-sc/data/'
victim='eddsa'

In [ ]:
onefiles = os.listdir(datadir + victim + '-1-parsed')
onelist = []
for fname in onefiles:
    with open(datadir + victim + '-1-parsed/' + fname, 'r') as f:
        onelist.append([int(n) for n in f])
print(len(onelist))
onethreshold = np.percentile(np.array([len(l) for l in onelist]),0.3)

zerofiles = os.listdir(datadir + victim + '-0-parsed')
zerolist = []
for fname in zerofiles:
    with open(datadir + victim + '-0-parsed/' + fname, 'r') as f:
        zerolist.append([int(n) for n in f])
print(len(zerolist))
zerothreshold = np.percentile(np.array([len(l) for l in zerolist]),0.3)
print((onethreshold, zerothreshold))

In [ ]:
threshold = int(min(zerothreshold, onethreshold))
newzero = [z for z in zerolist if len(z) >= threshold]
newone = [o for o in onelist if len(o) >= threshold]
dsize = min((len(newzero),len(newone)))
x_arr = np.zeros([dsize*2, threshold],dtype=np.float32)
y_arr = np.arange(dsize*2)%2
for i in range(dsize):
    x_arr[2*i] = newzero[i][:threshold]
    x_arr[2*i+1] = newone[i][:threshold]
print(x_arr.shape)

In [ ]:
std = 16
mean = np.median(x_arr)
x_arr2 = (x_arr-mean)/std
print((mean,))

In [ ]:
valcut = len(x_arr)*6 //10
testcut = len(x_arr) * 9 // 10
train_x = x_arr2[:valcut]
train_y = y_arr[:valcut]
valid_x = x_arr2[valcut:testcut]
valid_y = y_arr[valcut:testcut]
test_x = x_arr2[testcut:]
test_y = y_arr[testcut:]
from sklearn import svm
clf = svm.SVC(kernel='rbf',gamma='auto', C=1)

clf.fit(train_x, train_y)
pred_y = clf.predict(valid_x)
#tpred_y = clf.predict(train_x)
(pred_y == valid_y).sum()/len(pred_y)

In [ ]:
if os.path.isfile(victim + ".pkl"):
    x_arr_old, y_arr_old = pickle.load(open(victim + ".pkl",'rb'))
    newlen = min(x_arr_old.shape[1], x_arr.shape[1])
    print("Old tracelen {}, new tracelen {}".format(x_arr_old.shape[1], newlen))
    x_arr_new = np.concatenate([x_arr_old[:,:newlen], x_arr[:,:newlen]], axis=0)
    y_arr_new = np.concatenate([y_arr_old, y_arr])
else:
    x_arr_new = x_arr
    y_arr_new = y_arr
print("total num traces: " + str(len(y_arr_new)))
traincut = len(x_arr_new)*6 //10
valcut = len(x_arr_new) * 9 // 10
pickle.dump((x_arr_new, y_arr_new), open(victim + ".pkl", "wb"))
pickle.dump((x_arr_new[:traincut], y_arr_new[:traincut]), open(victim + "_train.pkl", "wb"))
pickle.dump((x_arr_new[traincut:valcut], y_arr_new[traincut:valcut]), open(victim + "_valid.pkl", "wb"))
pickle.dump((x_arr_new[valcut:], y_arr_new[valcut:]), open(victim + "_test.pkl", "wb"))

In [ ]:
valcut

